# Managing Log Files

Most recently I developed a centralized log system by aggregating log files in Syslog-ng and communicating the logs via Nginx-Lua websocket to a remote ElasticSearch server, where they are subject to text-search, re-indexing, visualization, and other analyses using Kibana.  This project highlights a method for avoiding Logstash, avoiding Syslog-ng's apparent persistent connection requirement, using relatively small amounts of resources, and providing low latency between log generation and analysis along a securable communcation channel.

My Linux stack generally includes Django, Celery, Redis, Postgresql, uWSGI, Supervisor, Nginx (OpenResty), Syslog-ng, ElasticSearch, and Kibana.  In developing the log management system, my primary goal was getting the log files into ElasticSearch, where each entry is distinct, indexable, text-searchable, and can be re-organized by any criteria.  This was accomplished by aggregating all log files.  Syslog-ng was chosen among other comprehensive log aggregation tools (syslog, rsyslog) for it's asynchronous, multi-threaded capacity. 

It is true that Syslog-ng 3.6 with the Syslog-ng Incubator package has the capacity to send logs directly to ElasticSearch and through a variety of channels (UDP, TCP, unix, Redis, and more).  One noteworthy limitation with Syslog-ng is the apparent need to maintain a persistent connection with a remote destination.  In my experience, communicating via TCP required an unbroken connection between a source and remote destination, which did not comport with Nginx style offloading.  Even idle connections had to be 'active' otherwise there were "Broken Connection" errors that brought Syslog-ng to a halt even though test messages could be readily communicated through the "Broken Connection."  And I found the same issue to exist when trying to use unix sockets in a series.  I avoided UDP due to concern of communication loss that is apparently inherent in the protocol.  The method provided herein circumvents Syslog-ng's limitation while expediting communication over websockets.



### (1) Getting the Log Files to Syslog-ng

Log information in this system was directed into several socket channels that are received by Syslog-ng. Conceptually, Supervisor and its processes share a log stream, and Nginx has a log stream.  


#### Supervisor:
When the service script (init.d on EC2) calls Supervisor, the command line options provide instruction for a log file:

> NAME="supervisord"
> DESC="supervisor_root"
> HOME="/home/ec2-user"
> SERVER="SERVER4"
> PIDFILE="$HOME/$SERVER/run/pids/sv_root.pid"
> LOGFILE="/var/log/syslogs/supervisor.log"
> DAEMON="/usr/bin/$NAME"
> DAEMON_OPTS="-c /etc/supervisor/conf.d/supervisord.conf \
> --user=0 \
> --umask=022 \
> --directory=$HOME/$SERVER \
> --logfile=$LOGFILE \
> --logfile_maxbytes=0 \
> --logfile_backups=0 \
> --loglevel=info \
> --pidfile=$PIDFILE \
> --identifier=$DESC \
> --strip_ansi"

Defining options via command-line, as opposed to using Supervisor's configuration file, has proven most consistent.  Still, a Supervisor configuration file was needed. LINK.

One benefit to Supervisor operating a series of processes: Supervisor's standard out, by default, includes the stdout and stderr of it's children.  Thus, once the Supervisor log was connected to Syslog-ng, most of the log files for child processes were also connected.

Syslog-ng is a great tool but I found it consumed unnecessary amount of resources at times and I noted it's sluggish response time in reading files on my dated hardware. Socat was used to transform the Supervisor log file into a socket stream as a qualitatively observed means of increasing Syslog-ng responsiveness.

    sudo socat -u exec:"/usr/bin/tail -n 30 -f /var/log/syslogs/supervisor.log",nonblock=1,ignoreeof,shut-null \
    unix:/var/sockets/supervisor.sock,crnl,ignoreeof,null-eof

I also limited Syslog-ng cpu usage with cpulimit LINK.


#### Nginx:
Logging for Nginx was handled separately, although it could be included with Supervisor.  Error and Access logs were again directed to sockets:

    log_format  gen_info  '{"ip": "$remote_addr",'
                            '"host": "$host",'
                            '"path": "$request_uri",'
                            '"status": "$status",'
                            '"referrer": "$http_referer",'
                            '"user_agent": "$http_user_agent",'
                            '"length": $bytes_sent,'
                            '"generation_time_milli": $request_time,'
                            '"date": "$time_iso8601"}';
    # (source for log format: http://forum.piwik.org/read.php?2,112779)

    error_log                           syslog:server=unix:/var/sockets/ngx_error.sock warn;
    access_log                          syslog:server=unix:/var/sockets/ngx_access.sock gen_info;
    
    
#### Delivered to Syslog-ng:
And Syslog-ng received the Supervisor and Nginx sockets:

source 		s_ngx_error	{ 		unix-dgram("/var/sockets/ngx_error.sock"
	 								max-connections(10)
	 								log-fetch-limit(20)
	 								log-iw-size(1000)										);	};
source 		s_ngx_access { 		unix-dgram("/var/sockets/ngx_access.sock"
	 								max-connections(10)
	 								log-fetch-limit(20)
	 								log-iw-size(1000)										);	};
source		s_supervisor {		unix-stream("/var/sockets/supervisor.sock"
	 								max-connections(10)
	 								log-fetch-limit(20)
	 								log-iw-size(1000)										);	};

### (2) Syslog-ng to Remote ElasticSearch

The `websocket_sender` config file acts as the websocket server, the `websocket_receiver` as the client.  The server in this case first resolves a DNS to an IP address.  The websocket communication (with no errors) begins with hand-shake, then information is communicated from server to client, and finally the client confirms receipt.  Here the handshake is simple, but it could involve more encrypted methods.  The client confirmation receipt provides the UID of the ElasticSearch document created from the server's communication, which, for instance, could be saved in Nginx's cache and be used to authenticate subsequent handshakes.

The missing piece here is what delivers the log entries to the `websocket_sender.`  As mentioned before, Syslog-ng seems to require persistent connection with it's destination, even when idle. One destination where there is no problem for Syslog-ng is a local file, so that's what I used.  All the log entries are formatted (to be addressed next), and then saved to a single log.  This script delivers the log entries:

    sudo tail -n 0 -f /var/log/syslogs/resty_logger.log | while read line
    do
        echo $line | socat - unix:/var/sockets/web_socket_relay.sock
    done

On the remote machine with 